# Data Preprocessing

## Importing the libraries

In [51]:
import numpy as np
import pandas as pd
from pprint import pprint

## Importing the dataset

In [52]:
dataset = pd.read_csv('Agaricus-lepiota.data.txt', header=None)

In [53]:
print(dataset.loc[dataset[11] == "?"])

     0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22
3984  e  x  y  b  t  n  f  c  b  e  e  ?  s  s  e  w  p  w  t  e  w  c  w
4023  p  x  y  e  f  y  f  c  n  b  t  ?  k  s  w  w  p  w  o  e  w  v  p
4076  e  f  y  u  f  n  f  c  n  h  e  ?  s  f  w  w  p  w  o  f  h  y  d
4100  p  x  y  e  f  y  f  c  n  b  t  ?  k  s  p  p  p  w  o  e  w  v  d
4104  p  x  y  n  f  f  f  c  n  b  t  ?  s  s  p  p  p  w  o  e  w  v  l
...  .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. ..
8119  e  k  s  n  f  n  a  c  b  y  e  ?  s  s  o  o  p  o  o  p  b  c  l
8120  e  x  s  n  f  n  a  c  b  y  e  ?  s  s  o  o  p  n  o  p  b  v  l
8121  e  f  s  n  f  n  a  c  b  n  e  ?  s  s  o  o  p  o  o  p  b  c  l
8122  p  k  y  n  f  y  f  c  n  b  t  ?  s  k  w  w  p  w  o  e  w  v  l
8123  e  x  s  n  f  n  a  c  b  y  e  ?  s  s  o  o  p  o  o  p  o  c  l

[2480 rows x 23 columns]


## Taking care of missing data

In [54]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values="?", strategy='most_frequent')
dataset = pd.DataFrame(imputer.fit_transform(dataset))

In [55]:
print(dataset.loc[dataset[11] == "?"])

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
Index: []


## Splitting the dataset into the Training set and Test set

In [56]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True)

train_data_index , test_data_index = next(kf.split(dataset), None)
train_data = dataset.iloc[train_data_index]
test_data = dataset.iloc[test_data_index]

print("\t\t\tTrain data \n" ,train_data ,"\n")
print("\t\t\tTest data \n" ,test_data)

			Train data 
      0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22
0     p  x  s  n  t  p  f  c  n  k  e  e  s  s  w  w  p  w  o  p  k  s  u
1     e  x  s  y  t  a  f  c  b  k  e  c  s  s  w  w  p  w  o  p  n  n  g
2     e  b  s  w  t  l  f  c  b  n  e  c  s  s  w  w  p  w  o  p  n  n  m
3     p  x  y  w  t  p  f  c  n  n  e  e  s  s  w  w  p  w  o  p  k  s  u
4     e  x  s  g  f  n  f  w  b  k  t  e  s  s  w  w  p  w  o  e  n  a  g
...  .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. ..
8118  p  k  y  n  f  f  f  c  n  b  t  b  k  s  p  w  p  w  o  e  w  v  d
8119  e  k  s  n  f  n  a  c  b  y  e  b  s  s  o  o  p  o  o  p  b  c  l
8121  e  f  s  n  f  n  a  c  b  n  e  b  s  s  o  o  p  o  o  p  b  c  l
8122  p  k  y  n  f  y  f  c  n  b  t  b  s  k  w  w  p  w  o  e  w  v  l
8123  e  x  s  n  f  n  a  c  b  y  e  b  s  s  o  o  p  o  o  p  o  c  l

[7311 rows x 23 columns] 

			Test data 
      0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 

## Functions

In [57]:
def entropy(target_col):

    #Calculate the entropy of a dataset.
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

In [58]:
elements,counts = np.unique(dataset[1],return_counts = True)


In [59]:
print(elements, counts, np.sum(counts))

['b' 'c' 'f' 'k' 's' 'x'] [ 452    4 3152  828   32 3656] 8124


In [60]:
def InfoGain(data, split_attribute_index, label_index = 0):
    """
    Calculate the information gain of a dataset. This function takes three parameters:
    1. data = The dataset for whose feature the IG should be calculated
    2. split_attribute_index = the index of the feature for which the information gain should be calculated
    3. label_index = label_index of the target feature.
    """    
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[label_index])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_index],return_counts=True)

    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_index]==vals[i]).dropna()[label_index]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [61]:
print(InfoGain(dataset, 5))

0.9060749773839998


In [62]:
def ID3(data, originaldata, features, label_index = 0, parent_node_class = None):
    """
    ID3 Algorithm: This function takes five paramters:
    1. data = the data for which the ID3 algorithm should be run --> In the first run this equals the total dataset
 
    2. originaldata = This is the original dataset needed to calculate the mode target feature value of the original dataset
    in the case the dataset delivered by the first parameter is empty

    3. features = the feature space of the dataset . This is needed for the recursive call since during the tree growing process
    we have to remove features from our dataset --> Splitting at each node

    4. label_index = the index of the target attribute

    5. parent_node_class = This is the value or class of the mode target feature value of the parent node for a specific node. This is 
    also needed for the recursive call since if the splitting leads to a situation that there are no more features left in the feature
    space, we want to return the mode target feature value of the direct parent node.
    """   
    #Define the stopping criteria --> If one of this is satisfied, we want to return a leaf node#
    
    #If all target_values have the same value, return this value
    if len(np.unique(data[label_index])) <= 1:
        return np.unique(data[label_index])[0]
    
    #If the dataset is empty, return the max value feature in the original dataset
    elif len(data)==0:
        return np.unique(originaldata[label_index])[np.argmax(np.unique(originaldata[label_index],return_counts=True)[1])]
    
    #If the feature space is empty, return the mode target feature value of the direct parent node --> Note that
    #the direct parent node is that node which has called the current run of the ID3 algorithm and hence
    #the mode target feature value is stored in the parent_node_class variable.
    
    elif len(features) ==0:
        return parent_node_class
    
    #If none of the above holds true, grow the tree!
    
    else:
        #Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = np.unique(data[label_index])[np.argmax(np.unique(data[label_index],return_counts=True)[1])]
        
        #Select the feature which best splits the dataset
        item_values = [InfoGain(data,feature,label_index) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        tree = {best_feature:{}}
        
        
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]
        
        #Grow a branch under the root node for each possible value of the root node feature
        
        for value in np.unique(data[best_feature]):
            value = value
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = data.where(data[best_feature] == value).dropna()
            
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
            subtree = ID3(sub_data,originaldata,features,label_index,parent_node_class)
            
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree
            
        return(tree)

In [63]:
def predict(query,tree,default = "p"):
    """
    Prediction of a new/unseen query instance. This takes two parameters:
    1. The query instance as a dictionary of the shape {"feature_name":feature_value,...}

    2. The tree 
    SUMMARIZED: If we have a query instance consisting of values for features, we take this features and check if the 
    name of the root node is equal to one of the query features.
    If this is true, we run down the root node outgoing branch whose value equals the value of query feature == the root node.
    If we find at the end of this branch a leaf node (not a dict object) we return this value (this is our prediction).
    If we instead find another node (== sub_tree == dict objct) we search in our query for the feature which equals the value 
    of that node. Next we look up the value of our query feature and run down the branch whose value is equal to the 
    query[key] == query feature value. And as you can see this is exactly the recursion we talked about
    with the important fact that for each node we run down the tree, we check only the nodes and branches which are 
    below this node and do not run the whole tree beginning at the root node 
    --> This is why we re-call the classification function with 'result'
    """
    
    
    #1.
    for key in list(query.keys()):
        if key in list(tree.keys()):
            #2.
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            #3.
            result = tree[key][query[key]]
            #4.
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result

In [64]:
def test(data,tree):
    #Create new query instances by simply removing the target feature column from the original dataset and 
    #convert it to a dictionary
    queries = data.iloc[:,1:].to_dict(orient = "records")

    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,"p") 
    return (np.sum(predicted["predicted"] == data[0].values )/len(data))*100

# Running the Code

In [65]:
accuracy = []
for i in range(10):
  train_data_index , test_data_index = next(kf.split(dataset), None)
  train_data = dataset.iloc[train_data_index].reset_index(drop=True)
  test_data = dataset.iloc[test_data_index].reset_index(drop=True)
  features = train_data.columns[1:]
  tree = ID3(train_data, train_data, features )
  accuracy.append(test(test_data,tree))
  
pprint(tree)
print("\nAverage accuracy of 10 runs of program" , u"\u00B1 STD = %.2f%%" % np.mean(accuracy), u"\u00B1 %.2f" % np.std(accuracy))

{5: {'a': 'e',
     'c': 'p',
     'f': 'p',
     'l': 'e',
     'm': 'p',
     'n': {20: {'b': 'e',
                'h': 'e',
                'k': 'e',
                'n': 'e',
                'o': 'e',
                'r': 'p',
                'w': {22: {'d': {8: {'b': 'e', 'n': 'p'}},
                           'g': 'e',
                           'l': {3: {'c': 'e', 'n': 'e', 'w': 'p', 'y': 'p'}},
                           'p': 'e',
                           'w': 'e'}},
                'y': 'e'}},
     'p': 'p',
     's': 'p',
     'y': 'p'}}

Average accuracy of 10 runs of program ± STD = 100.00% ± 0.00
